# Логистическая регрессия

Обучим логистическую регрессию для предсказания того, откликнется клиент
на рекламное предложение (target = 1) или нет (target = 0).

## Подлючение библиотек, загрузка и обзор данных

In [1]:
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd

import warnings
warnings.filterwarnings("ignore")

In [2]:
data = pd.read_csv('https://raw.githubusercontent.com/evgpat/edu_stepik_practical_ml/main/datasets/clients_data.csv')

In [3]:
data.head()

,AGE,SOCSTATUS_WORK_FL,SOCSTATUS_PENS_FL,GENDER,CHILD_TOTAL,DEPENDANTS,PERSONAL_INCOME,LOAN_NUM_TOTAL,LOAN_NUM_CLOSED,LOAN_DLQ_NUM,TARGET
0,49,1,0,1,2,1,5000.0,1,1,2,0
1,32,1,0,1,3,3,12000.0,1,1,1,0
2,52,1,0,1,4,0,9000.0,2,1,0,0
3,39,1,0,1,1,1,25000.0,1,1,3,0
4,30,1,0,0,0,0,12000.0,2,1,2,0


In [4]:
from sklearn.model_selection import train_test_split

X = data.drop('TARGET', axis=1)
y = data['TARGET']

Xtrain, Xtest, ytrain, ytest = train_test_split(X, y, train_size=0.7, random_state=123)

## Практика

**Задание**
Выведите на экран количество объектов каждого класса. Сколько процентов объектов относятся к положительному классу?
Ответ округлите до целого числа (например, если доля объектов положительного класса равна 0.412, в ответ запишите 41,
имея в виду 41 процент).

In [9]:
# ваш код здесь
round((y.value_counts()[1] / len(y)) * 100)

12

Обучим логистическую регрессию с параметрами по умолчанию.

In [10]:
from sklearn.linear_model import LogisticRegression

lr = LogisticRegression()

In [11]:
lr.fit(Xtrain, ytrain)

prediction = lr.predict(Xtest)

In [15]:
pd.Series(prediction).value_counts()

0    4567
dtype: int64

Метрику accuracy не стоит использовать при сильном дисбалансе классов. Поэтому посчитайте f1_score для оценки качества модели на тестовых данных.

f1_score принимает значения от 0 до 1. Чем ближе к 1, тем лучше модель.

In [16]:
from sklearn.metrics import f1_score, accuracy_score, precision_score, recall_score

print('f1:', f1_score(ytest, prediction))
print('acc:', accuracy_score(ytest, prediction))
print('pre:', precision_score(ytest, prediction))
print('rec:', recall_score(ytest, prediction))

f1: 0.0
acc: 0.8776001751696957
pre: 0.0
rec: 0.0


**Вопрос**
Чему равен `f1_score`?

Удивительно, да?

Давайте разберемся, почему качество такое низкое.

Предскажем вероятности классов с помощью обученной логистической регрессии на тестовых данных.

In [62]:
probs_test = lr.predict_proba(Xtest)

probs_test.min(axis=0)

array([0.65876604, 0.02628033])

По вероятностям видно, что вероятности отнесения к положительному классу очень низкие.

Попробуем изменить порог для перевода вероятности в классы.

**Вопрос**
Чему равен `f1_score`, если все объекты с вероятностью не меньшей 0.1, относить к положительному классу?

Ответ округлите до сотых.

In [63]:
# ваш код здесь
prediction_10 = np.where(probs_test >= 0.1, 1, 0)[:, 1]
prediction_10


array([1, 1, 1, ..., 0, 0, 0])

In [64]:
print('f1:', f1_score(ytest, prediction_10))
print('acc:', accuracy_score(ytest, prediction_10))
print('pre:', precision_score(ytest, prediction_10))
print('rec:', recall_score(ytest, prediction_10))

f1: 0.2469521725539231
acc: 0.4725202539960587
pre: 0.14962121212121213
rec: 0.7066189624329159


In [65]:
round(f1_score(ytest, prediction_10), 2)

0.25

У обученной модели можно посмотреть веса (как и в линейной регрессии).
Выведем на экран веса модели (`model.coef_`, `model.intercept_`).

In [72]:
lr.coef_[0], lr.intercept_

(array([-5.55214705e-02, -7.90868782e-04, -4.28172545e-04, -1.07248154e-03,
        -1.34958177e-03, -4.95807875e-04,  1.13131722e-05, -1.80011616e-03,
        -1.26843809e-03, -1.45283567e-05]),
 array([-0.00115086]))

In [73]:
coefs = pd.DataFrame({'feature': X.columns, 'coefs': lr.coef_[0]})
coefs

,feature,coefs
0,AGE,-0.055521
1,SOCSTATUS_WORK_FL,-0.000791
2,SOCSTATUS_PENS_FL,-0.000428
3,GENDER,-0.001072
4,CHILD_TOTAL,-0.001350
5,DEPENDANTS,-0.000496
6,PERSONAL_INCOME,0.000011
7,LOAN_NUM_TOTAL,-0.001800
8,LOAN_NUM_CLOSED,-0.001268
9,LOAN_DLQ_NUM,-0.000015


In [78]:
coefs.sort_values('coefs')

,feature,coefs
0,AGE,-0.055521
7,LOAN_NUM_TOTAL,-0.001800
4,CHILD_TOTAL,-0.001350
8,LOAN_NUM_CLOSED,-0.001268
3,GENDER,-0.001072
1,SOCSTATUS_WORK_FL,-0.000791
5,DEPENDANTS,-0.000496
2,SOCSTATUS_PENS_FL,-0.000428
9,LOAN_DLQ_NUM,-0.000015
6,PERSONAL_INCOME,0.000011


Создайте `pd.DataFrame`, где в первом столбце стоят названия признаков, а во втором - их веса (так удобнее анализировать результат).
Отсортируйте таблицу по убыванию весов.

**Вопрос**
Какой признак имеет наибольший положительный вес?

In [ ]:
# ваш код здесь

## Бонус

**Задание 1**
Подберите порог для перевода вероятностей в классы, дающий максимальное значение `f1_score`.

In [ ]:
# ваш код здесь

**Задание 2**
Во вложенном цикле подберите одновременно коэффициент регуляризации `C` у логистической регрессии и порог для перевода вероятностей в классы, дающие максимальное значение `f1_score`.

По-хорошему, чтобы не переобучиться, эти величины надо подбирать не по тесту, а по отдельной выборке.

Поэтому разобъем данные изначально на три части: `Xtrain`, `Xval`, `Xtest`.

*   В цикле при подборе `С` и порога будем обучаться по `Xtrain`, а предсказывать и измерять качество по `Xval`.

*   Качество итоговой модели с найденными `C` и порогом измерьте по `Xtest`.

Так не переобучимся!


In [79]:
Xtrain_new, Xval, ytrain_new, yval = train_test_split(Xtrain, ytrain, train_size=0.7, random_state=123)

# ваш код для подбора C и порога здесь

In [84]:
params = {'C': [0.01, 0.1, 1, 10, 100, 1000, 10000], 'prob': [0.05, 0.1, 0.15, 0.20, 0.25, 0.30, 0.35]}
best_params = {'C': 0, 'prob': 0, 'f1': -1}
for C in params['C']:
    for prob in params['prob']:
        model = LogisticRegression(C=C)
        model.fit(Xtrain, ytrain) # обучаемся на всех тренировочных данных
        prediction = model.predict_proba(Xval)
        classes = np.where(prediction >= prob, 1, 0)[:, 1] # переведите полученные вероятности в классы по найденному порогу
        f1 = f1_score(yval, classes)
        if f1 > best_params['f1']:
            best_params['C'] = C
            best_params['prob'] = prob
            best_params['f1'] = f1
        print(f'C: {C}, prob: {prob}, f1: {f1}')
print(best_params)


C: 0.01, prob: 0.05, f1: 0.21716848174247275
C: 0.01, prob: 0.1, f1: 0.22916666666666666
C: 0.01, prob: 0.15, f1: 0.2192691029900332
C: 0.01, prob: 0.2, f1: 0.2042175360710322
C: 0.01, prob: 0.25, f1: 0.07459207459207459
C: 0.01, prob: 0.3, f1: 0.0056022408963585435
C: 0.01, prob: 0.35, f1: 0.0
C: 0.1, prob: 0.05, f1: 0.21723806472284526
C: 0.1, prob: 0.1, f1: 0.22927050294517445
C: 0.1, prob: 0.15, f1: 0.2192691029900332
C: 0.1, prob: 0.2, f1: 0.2042175360710322
C: 0.1, prob: 0.25, f1: 0.07459207459207459
C: 0.1, prob: 0.3, f1: 0.0056022408963585435
C: 0.1, prob: 0.35, f1: 0.0
C: 1, prob: 0.05, f1: 0.21723806472284526
C: 1, prob: 0.1, f1: 0.22927050294517445
C: 1, prob: 0.15, f1: 0.2192691029900332
C: 1, prob: 0.2, f1: 0.2042175360710322
C: 1, prob: 0.25, f1: 0.07459207459207459
C: 1, prob: 0.3, f1: 0.0056022408963585435
C: 1, prob: 0.35, f1: 0.0
C: 10, prob: 0.05, f1: 0.21723806472284526
C: 10, prob: 0.1, f1: 0.22927050294517445
C: 10, prob: 0.15, f1: 0.2192691029900332
C: 10, prob: 

In [85]:
model = LogisticRegression(C=best_params['C'])
model.fit(Xtrain, ytrain)
prediction = model.predict_proba(Xtest)
classes = np.where(prediction >= best_params['prob'], 1, 0)[:, 1]
f1_test = f1_score(ytest, classes)

In [86]:
# вычислите значение f1_score на тестовых данных
round(f1_test, 2)

0.25